## Reinforcement learning - Disrete and Continuous maps with Q-learning

In the previous blog, we got introduced to the concept of reinforcement learning through an example (MountainCar-v0) of OpenAI's gym. The problem was to build a model that will decide the action taken by a Mountain car such that the car reaches its target -  a flag. Each state of the car has two variables representing instantaneous distance and velocity of the car. The target was located at a distance $s=0.5$.

We built a model by running $10^4$ sessions. The actions taken at various states were random. And we selected the top $1\%$ of the most rewarding sessions that had maximum $s$ close to $0.5$, as the training set. The model was built using neural network with two hidden layers. The actions determined by the model were evaluated to be successful.

From this exercise, the questions that can bother us are: 
- If we are to run so many number of sessions to build a model, we have to store the states and outcomes. What if this leads to problems related to memory?
- What if running each session is expensive in an application such as medical diagnosis or a space rover that will have to learn its best actions on the run? 

### Alternative approach

To address the above questions let us re-examine our approach. We know that the rewarding action is something that can move the car closer to the target $s=0.5$. The best action in a particular state results in maximum displacement of the car from its current state. The measure of reward $r$ is therefore set as $|s_{old}-s_{new}|$. 

Let us define a quantity, $Q$ that represents expected future rewards from a session. This expected future rewards $Q$ has to be maximized.

We will require some hyperparameters for the following reasons:

- If the car keeps oscillating about a point, the expected future rewards will keep increasing. This can lead to infinite sum and $Q\rightarrow \infty$. To avoid this we need to include something that can make infinite sums finite. A geometric parameter $\gamma(<1)$ will be of help in this situation.

- Since the model keeps learning it is wise to give more value to recent steps when compared to past computation. Therefore, a learning step parameter $\alpha$ is introduced.

With these hyperparamters, the desired equation is:

$$Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \alpha \big[r_t + \gamma \max\limits_{a} Q(s_{t+1},a_{t+1}) - Q(s_t,a_t)\big]$$

The required condition is $Q\rightarrow r+\gamma \max Q,$ which is the sum of current reward and the maximum of expected discounted future rewards. This condition can be met iteratively. $Q$ is introduced formally through Bellman optimality equation.

### Two ways of applying Q-learning

Let us apply the above approach in two ways to make the car reach the flag.

- We can discretize the state and build a q-table. In this problem, states with negative and positive velocities are categorized as two separate states. We already know that there are three actions in this Mountain car example. The car may move left, right or it may stop. Three actions and two categories of state will result in a $Q$-table of dimension 2x3.

- We can build a neural network as a regression model that can accomodate infinitely many states. This will take a continous set of floating point values and provide $Q$, which will be maximum for the appropriate action. Therefore, the index of the maximum will provide us the desired action.

One more thing we should keep in mind. In this reinforcement learning approach we are trying to learn as we generate data. The model has to make random choices before learning to make right decisions. This can be achieved by introducing an exploration-exploitation constant.

### With a Q-table

The states are discretized into two parts. One with a positive velocity and the other with a negative velocity. A Q-table that provides values at two states with rewards from three actions (left, stop and right). We initialize the table fo dimension 2x3 with zeros.

In [119]:
import gym
import numpy as np
env = gym.make('MountainCar-v0')
def discretize(s):
    if s[1]<0:
        return 0
    return 1
def q_learning_table(env, num_episodes=100):
    q_table = np.zeros((2,3))
    gamma = 0.95 # discount rate
    lr = 0.1 # learning rate
    eps = 0.01 #exploration-exploitation constant
    for i in range(num_episodes):
        s = env.reset()
        done = False
        while not done:
            if np.sum(q_table[discretize(s),:]) == 0 or np.random.random()<eps:
                a = np.random.randint(0, 3)
            else:
                a = np.argmax(q_table[discretize(s), :])
            new_s, r, done, _ = env.step(a)
            def_reward = abs(s[0]-new_s[0]) # define a new reward
            q_table[discretize(s), a] += def_reward + lr*(gamma*np.max(q_table[discretize(new_s), :]) - q_table[discretize(s), a])
            s = new_s
    return q_table
def eval_model(q_table, numevals=10):
    c = 0
    for _ in range(numevals):
        s = env.reset()
        done = False
        while not done:
            a = np.argmax(q_table[discretize(s)])
            s,r,done,_ = env.step(a)
            if s[0]>=0.5:            
                c+=1
                break
    return c
numevals = 10
num_episodes = 1000
q_table = q_learning_table(env, num_episodes)
print('Reached the flag in {}/{} games'.format(eval_model(q_table),numevals))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Reached the flag in 10/10 games


### With a Q-model

In case we face a situation where we are unable to discretize the states, then we can build a regression model instead of a table. Regression models are continuous maps of the input. It may take more number of episodes to optimize the model and we will use the same two leayer neural network as in the previous blog.

As the model optimizes (or learns) the amount of exploration can be reduced. This is done by introducing a decay factor in the code.

In [121]:
import gym
import numpy as np
import random
from matplotlib import pyplot as plt
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation
from keras.optimizers import Adam
import matplotlib.pyplot as plt

env = gym.make('MountainCar-v0')
class dqnn(gym.Wrapper):
  def __init__(self, env, states = 2, actions = 3, gamma = 0.99, eps = 0.5, 
               lr = 0.001, episodes = 3000, decay = 0.999):
    gym.Wrapper.__init__(self, env)
    self.states = states
    self.actions = actions
    self.gamma = gamma
    self.eps = eps
    self.learning_rate = lr
    self.num_episodes = episodes
    self.decay_rate = decay
    self.model = self.nn_model()
  def nn_model(self):
    model = Sequential()
    model.add(Dense(256, input_dim=self.states, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(self.actions, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr= self.learning_rate))
    return model
  def qlearn_to_play(self):
    steps_list=[]
    print('Total episodes:', self.num_episodes)
    for i in range(self.num_episodes):
      j=0   
      s = self.env.reset()
      done = False
      self.eps *= self.decay_rate
      while not done:
          j+=1
          Q = self.model.predict(s.reshape(-1, len(s)))
          if np.random.random() < self.eps:
              a = np.random.randint(0, 3)
          else:
              a = np.argmax(Q)
          new_s, r, done, _ = self.env.step(a)
          Q1 = self.model.predict(new_s.reshape(-1, len(s)))
          target = abs(s[0]-new_s[0]) + self.gamma * np.max(Q1)
          Q[0][a] = target
          self.model.fit(s.reshape(-1, len(s)), Q.reshape(-1, 3), epochs=1, verbose=0)
          s = new_s
      print('Closing epsiode {} in {} steps'.format(i+1,j))
      steps_list.append(j)
    return steps_list
  def eval_model(self,numevals=10):
    c = 0
    for _ in range(numevals):
        s = self.env.reset()
        done = False
        while not done:
            a = np.argmax(self.model.predict(s.reshape(-1, len(s)))[0])
            s,r,done,_ = self.env.step(a)
            if s[0]>=0.5:            
                c+=1                
    print('Model is {}% accurate.'.format(c/numevals*100))
      
dql = dqnn(env)  
steps_list = dql.qlearn_to_play()
dql.eval_model()
plt.plot(steps_list,'.')
plt.xlabel('Number of training episodes')
plt.ylabel('Number of steps to reach the flag')

As the number of training episodes increases, the target is reached in more number of episodes. This can be considered as an evidence that our model is converging with the number of episodes.

<img src="steps_episodes_v2.png"/>

### Conclusions

The following points may help us in designing a reinforcement problem:

- In the problem at have do we have a suitable reward defined? For example, in this exercise the reward provided by OpenAI's gym is -1 till the car reaches the flag. This is not suitable to build a model. Hence, it was modified as the displacement from the current state.

- Is it possible to generate a training set without having any constraints of memory and computational resources? Generating a training set and then building a model is fairly easier but in most of the situations this is going to be expensive.

- Is it possible to discretize the states and build a table for choosing best action? With the right reward setup, building a table can be more efficient than using a regression model.

- If we are unable to discretize the states then we have to build a regression model. Convergence of the model with suitable hyperparameters can sometimes be challenging.